## **06: Retrieval Strategies**

In [1]:
from dotenv import load_dotenv
from pathlib import Path
load_dotenv()
print("✅ Setup complete")
import os

✅ Setup complete


### **Creating a Retriever**

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

docs = [
    Document(page_content="LangChain is a framework for LLM applications", metadata={"topic": "langchain"}),
    Document(page_content="RAG combines retrieval with generation", metadata={"topic": "rag"}),
    Document(page_content="Vector databases store embeddings", metadata={"topic": "vectors"}),
    Document(page_content="Transformers use attention mechanisms", metadata={"topic": "transformers"}),
    Document(page_content="FAISS is a similarity search library", metadata={"topic": "vectors"}),
]
vector_store = FAISS.from_documents(docs, embeddings)
print("✅ Vector store created")

✅ Vector store created


In [4]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)
query = "How does RAG works?"
result = retriever.invoke(query)

print(f"Query: {query}\n")
print("Results:")
for i, doc in enumerate(result,1):
    print(f"{i}. {doc.page_content}")
    print(f"Topic: {doc.metadata['topic']}\n")

Query: How does RAG works?

Results:
1. RAG combines retrieval with generation
Topic: rag

2. Transformers use attention mechanisms
Topic: transformers

3. LangChain is a framework for LLM applications
Topic: langchain



### **MMR (Maximum Marginal Relevance)**

In [5]:
mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k":3,
        "fetch_k":5,
        "lambda_mult":0.7,
    }
)
query = "vector database"
print(f"Query: {query}\n")

print("Similarity Search:")
sim_result = retriever.invoke(query)
for doc in sim_result:
    print(f"   - {doc.page_content}")

print("\nMMR Search (more diverse):")
mmr_result = mmr_retriever.invoke(query)
for doc in mmr_result:
    print(f"   - {doc.page_content}")

Query: vector database

Similarity Search:
   - Vector databases store embeddings
   - RAG combines retrieval with generation
   - LangChain is a framework for LLM applications

MMR Search (more diverse):
   - Vector databases store embeddings
   - LangChain is a framework for LLM applications
   - RAG combines retrieval with generation


In [7]:
from langchain_core.runnables import chain

@chain
def custom_retriever(query:str):
    result = vector_store.similarity_search(query, k=5)
    filtered = [doc for doc in result if doc.metadata['topic']=='vectors']
    return filtered[:2]
query = "search_technology"
result = custom_retriever.invoke(query)

print(f"Custom Retriever Results for: {query}\n")
for doc in result:
    print(f"   +{doc.page_content}")
    print(f"   -{doc.metadata}")
    print()

Custom Retriever Results for: search_technology

   +FAISS is a similarity search library
   -{'topic': 'vectors'}

   +Vector databases store embeddings
   -{'topic': 'vectors'}



In [8]:
query = "Langchain framework"
result_with_score = vector_store.similarity_search_with_score(query, k=3)

print(f"Query: {query}\n")
print("Results with scores:")
for doc, score in result_with_score:
    print(f"  Score: {score:.4f}")
    print(f"  Content: {doc.page_content}\n")

Query: Langchain framework

Results with scores:
  Score: 0.1146
  Content: LangChain is a framework for LLM applications

  Score: 0.5073
  Content: FAISS is a similarity search library

  Score: 0.5287
  Content: Transformers use attention mechanisms

